In [1]:
import pandas as pd


In [3]:
data = pd.read_csv('../data/fama_term_structure.csv')
data.describe()

,KYTREASNOX,TTERMTYPE,RMTREASNO,RMCRSPID,TMDURATN,TMBID,TMBIDRET,TMBIDYLD,TMBIDFWD,TMASK,TMASKRET,TMASKYLD,TMASKFWD,TMNOMPRC,TMAVERET,TMAVEYLD,TMAVEFWD
count,1.150000e+03,1150.000000,1080.000000,1.079000e+03,1079.000000,1079.000000,1061.000000,1079.000000,1079.000000,1079.000000,1061.000000,1079.000000,1079.000000,1079.000000,1061.000000,1079.000000,1079.000000
mean,2.000021e+06,1449.873913,207642.250926,2.022525e+07,161.531974,98.886046,0.001988,0.002115,0.002167,98.893582,0.001978,0.002100,0.002155,98.889814,0.001983,0.002107,0.002161
std,1.137120e+01,2772.492929,6329.266241,1.491870e+04,99.121085,1.319480,0.002196,0.001897,0.001896,1.315816,0.002206,0.001893,0.001896,1.317638,0.002200,0.001895,0.001895
min,2.000001e+06,1.000000,0.000000,2.020023e+07,7.000000,94.681250,-0.003879,-0.000013,-0.000194,94.696667,-0.004116,-0.000021,-0.000118,94.688958,-0.003998,-0.000017,-0.000156
25%,2.000014e+06,605.000000,207633.000000,2.021091e+07,83.000000,98.101361,0.000074,0.000084,0.000097,98.108444,0.000064,0.000074,0.000089,98.106135,0.000069,0.000078,0.000098
50%,2.000019e+06,1204.000000,207830.000000,2.022123e+07,150.000000,99.502278,0.001250,0.001993,0.002239,99.505056,0.001320,0.001985,0.002208,99.503417,0.001277,0.001989,0.002253
75%,2.000024e+06,1209.000000,208041.000000,2.024032e+07,234.000000,99.957500,0.004198,0.004085,0.004079,99.963368,0.004176,0.004061,0.004076,99.959625,0.004177,0.004071,0.004076
max,2.000090e+06,26182.000000,208331.000000,2.025123e+07,370.000000,100.001125,0.008399,0.004572,0.005759,100.001875,0.008609,0.004563,0.005571,100.001500,0.008504,0.004568,0.005592
